In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.special import roots_hermite
from scipy.optimize import fmin_slsqp as min

import matplotlib.pyplot as plt
import math
import pickle

from scipy.interpolate import CubicSpline

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_CarrWu_strikes, generate_mthly_strikes

# #Import pricing functions
# from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec, BSM_put_vec

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gamma_with_spot

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [22]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
prod_type = "PE"

# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019:[8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}


#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1



In [23]:
# def Hermitetuple(no_of_options):
#     return pd.DataFrame(roots_hermite(no_of_options)).transpose()

# def findStrikes(x):
#     return K*np.exp(x*sigma*np.sqrt(2*(T-t))+(delta-r-(sigma**2)/2)*(T-t))

# def findWeights(x):
#     return (np.exp(-delta*(T-t))/np.sqrt(np.pi))*x   

# def portfolioStrikes(no_of_options):
#     return pd.Series(Hermitetuple(no_of_options)[0].apply(findStrikes))

# def portfolioWeights(no_of_options):
#     return pd.Series(Hermitetuple(no_of_options)[1].apply(findWeights))

def generate_CarrWu_avail_Strikes(aa, bb):
    aa_nearest = np.array(bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)])
    
    if (len(np.unique(aa_nearest)) < len(aa_nearest)):
        print("There are repititions in the CarrWu strikes")
        
    return aa_nearest


In [24]:
#########################
#User-defined functions #
#########################

def Hermitetuple(no_of_options):
    return pd.DataFrame(roots_hermite(no_of_options)).transpose()

def cubic_func(x, a0, a1, a2, a3):
    y = a0 + a1*x + a2*(np.square(x)) + a3 * np.power(x, 3)
    return y

def generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, \
                                       dict_wkly_expiries_each_month, \
                                       dict_ce_wkly_strikes, dict_pe_wkly_strikes, \
                                       dict_wkly_spots, \
                                       prod_type, prod_moneyness, output_path, \
                                       option_type = "CE", no_of_assets=1, cor_mat=[[1]],\
                                       batch_size_divisor=10, no_of_epochs = 1000, stock_ident="BANKNIFTY"):
    ce_pfl_weights = []
    pe_pfl_weights = [] 
    cash_weights = []
    cubic_polyn_volSurf = pd.read_csv(output_path + "A3_" + stock_ident + "_cubicPolyn_vol_surface_params.csv")

    for each_month in mthly_expiries_list:
        strike = dict_mthly_strikes[each_month.strftime("%d-%b-%Y")]
        weekly_pfl_weights = []
        ce_dict = {}
        pe_dict = {}
        cash_dict = {}
        week_list = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")]
        no_of_weeks = len(week_list)
        
        for week in range(0, no_of_weeks):
            s_t = dict_wkly_spots[each_month.strftime("%d-%b-%Y")][week] 
            K = strike
            r = np.array([dict_wkly_ir[each_month.strftime("%d-%b-%Y")][week]])[0]
            
            if (week == no_of_weeks-1):
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = each_month
            else:
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week + 1]
              
            df_polyn_date = cubic_polyn_volSurf[cubic_polyn_volSurf['Date'] == sim_start_date.strftime("%d-%b-%Y")]

            # We define Moneyness as (S/K) in vol surface for both calls and puts
            option_moneyness = float(s_t/float(K))
    
            # Need higher tenor first
            t_list = -np.sort(np.unique(np.array(-df_polyn_date['T'])))
            tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]
            print(each_month, " Week: ", week)
            print(tenor_list)
            
            if (len(tenor_list) > 2):
                print("There are more than 2 tenors in the vol surface - Please check!")
            
            df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[0]]
            polyn_coeff = df_polyn_coeff[["a0", "a1", "a2", "a3"]].to_numpy().reshape(-1)

            x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)

            if (option_moneyness <= x_min):
                vol_list = [cubic_func(x_min, *polyn_coeff)]
            elif (option_moneyness >= x_max):
                vol_list = [cubic_func(x_max, *polyn_coeff)]
            else:
                vol_list = [cubic_func(option_moneyness, *polyn_coeff)]
            
            vol = vol_list[0]
            dt = float((each_month - sim_end_date).days) / 365
    
            # r =risk_free rate - divid -> as obtained from futures: r = rf - q
#             risk_free_rate = 0.05
            risk_free_rate = get_risk_free_rate_from_exact_date(sim_start_date)

            #dividend
            q = risk_free_rate - r
            
            #Gaussian Quadrature - Strikes and weights   
            find_strikes = lambda x:  K*np.exp(x*vol*np.sqrt(2*(dt))+(-r-(vol**2)/2)*(dt))
            find_weight = lambda x: (np.exp(-q*(dt))/np.sqrt(np.pi))*x 
            generate_hermite_table = lambda no_of_calls: pd.DataFrame(roots_hermite(no_of_calls)).transpose()
            
            #Generate theoretical strikes and weights
            no_of_calls=10
            hermit_table = generate_hermite_table(no_of_calls=no_of_calls)
            call_theoritical_strikes = np.array(pd.Series(hermit_table[0].apply(find_strikes)))
            ce_wt_list = pd.Series(hermit_table[1].apply(find_weight))
            
            ce_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_calls)]).reshape(-1)

            print("Theoeretical Strikes: ")
            print(call_theoritical_strikes)  
            call_available_strikes = np.sort(np.array(dict_ce_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(-1)
            print("Market Liquid Available Strikes: ")
            print(call_available_strikes)
            call_strikes = generate_CarrWu_avail_Strikes(call_theoritical_strikes, call_available_strikes)
            print("Final CarrWu Strikes: ")
            print(call_strikes)
            ce_dict = {"Date": ce_dates_list, "Strike": call_strikes, "Pfl_weights": ce_wt_list}
            df_ce = pd.DataFrame(ce_dict)
            df_ce = df_ce.groupby(['Date', 'Strike'])[['Pfl_weights']].sum().reset_index()
        
            ce_pfl_weights.append(df_ce)
 
            # Put options
            put_strikes = [0]
            pe_wt_list = [0]

            #Put Pfl Weights
            pe_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, 1)]).reshape(-1)
            pe_dict = {"Date": pe_dates_list, "Strike":put_strikes, "Pfl_weights": pe_wt_list}
            df_pe = pd.DataFrame(pe_dict)
            pe_pfl_weights.append(df_pe)
            
            # Cash component
            cash_date = [sim_start_date.strftime("%d-%b-%Y")]
            cash_dict = {"Date":cash_date, "Pfl_weights":[0]} 
            df_cash = pd.DataFrame(cash_dict)
            cash_weights.append(df_cash)
                   
        df_ce_pfl_weights = pd.concat(ce_pfl_weights, axis=0)
        df_pe_pfl_weights = pd.concat(pe_pfl_weights, axis=0)
        df_cash_pfl_weights = pd.concat(cash_weights, axis=0)

        df_ce_pfl_weights.to_csv(output_path  + "B0B_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv", index = False)
        df_pe_pfl_weights.to_csv(output_path + "B0B_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv", index = False)
        df_cash_pfl_weights.to_csv(output_path  + "B0B_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv", index = False)
    
    return df_ce_pfl_weights, df_pe_pfl_weights, df_cash_pfl_weights

def output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_path):
    list_of_dates = []
    list_of_prod_type = []
    list_of_strikes = []
    for each_month in dict_mthly_strikes.keys():
        list_of_dates.append(each_month)
        list_of_strikes.append(dict_mthly_strikes[each_month])
        list_of_prod_type.append(prod_type)
    dict_hedged_prod = {"Date":list_of_dates, "prod_type":list_of_prod_type, "Strike":list_of_strikes}
    df_hedged_prod = pd.DataFrame(dict_hedged_prod) 
    df_hedged_prod.to_csv(output_path + "B0B_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv", index = False)
    return(df_hedged_prod)


In [25]:

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

# #Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

# #Load weekly strikes of short term options used for hedging monthly options 
dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots = generate_weekly_CarrWu_strikes(dict_wkly_expiries_each_month, input_data_path, stock_ident = stock_ident)     

#Load monthly strikes of interest as of required moneyness on the begining of month
dict_mthly_strikes = generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest - now kept it as cosntant
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)

# #Load Static portfolio weights built by neural network
ce_pfl_wts, pe_pfl_wts, cash_pfl_wts = generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, dict_wkly_expiries_each_month, \
                                                        dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots, \
                                                        prod_type, prod_moneyness, output_data_path, \
                                                        option_type = prod_type, no_of_assets=no_of_assets, \
                                                        cor_mat=cor_mat, stock_ident=stock_ident) 

# The strike correspnding to the option that is hedged
df_hedged_prod = output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_data_path)



2019-08-29  Week:  0
['35D', '7D']
Theoeretical Strikes: 
[ 7720.33708237  8253.28276302  8740.36672236  9217.99298511
  9702.80064066 10207.14106738 10743.97160509 11331.08690217
 11999.8136174  12828.17755641]
Market Liquid Available Strikes: 
[11200 11250 11300 11350 11400 11450 11500 11550 11600 11650 11700 11800
 11900 12000 12500]
There are repititions in the CarrWu strikes
Final CarrWu Strikes: 
[11200 11200 11200 11200 11200 11200 11200 11350 12000 12500]
2019-08-29  Week:  1
['28D', '7D']
Theoeretical Strikes: 
[ 7917.03054147  8410.63699643  8859.14854656  9296.67471487
  9738.60589056 10196.1463303  10680.8352189  11208.32890332
 11806.03214184 12542.10782605]
Market Liquid Available Strikes: 
[11000 11050 11100 11150 11200 11250 11300 11350 11400 11500 11700]
There are repititions in the CarrWu strikes
Final CarrWu Strikes: 
[11000 11000 11000 11000 11000 11000 11000 11200 11700 11700]
2019-08-29  Week:  2
['21D', '6D']
Theoeretical Strikes: 
[ 7899.11441324  8397.37342385 

2020-01-30  Week:  0
['35D', '7D']
Theoeretical Strikes: 
[ 8705.50294699  9244.0060979   9733.10334173 10210.03915979
 10691.60307254 11190.00066265 11717.78517145 12291.97320382
 12942.33735888 13742.92171228]
Market Liquid Available Strikes: 
[11900 12000 12100 12150 12200 12250 12300 12350 12400 12450 12500 12600
 12700 12800 13000]
There are repititions in the CarrWu strikes
Final CarrWu Strikes: 
[11900 11900 11900 11900 11900 11900 11900 12300 13000 13000]
2020-01-30  Week:  1
['28D', '7D']
Theoeretical Strikes: 
[ 8702.15048966  9243.95679901  9736.22767181 10216.409113
 10701.39422048 11203.47816864 11735.3206197  12314.095414
 12969.8611893  13777.38027693]
Market Liquid Available Strikes: 
[12100 12150 12200 12250 12300 12350 12400 12450 12500 12550 12600 12700
 12800 13000]
There are repititions in the CarrWu strikes
Final CarrWu Strikes: 
[12100 12100 12100 12100 12100 12100 12100 12300 13000 13000]
2020-01-30  Week:  2
['21D', '7D']
Theoeretical Strikes: 
[ 8734.18176845 

Theoeretical Strikes: 
[8900. 8900. 8900. 8900. 8900. 8900. 8900. 8900. 8900. 8900.]
Market Liquid Available Strikes: 
[ 8000  8500  8600  8700  8800  8900  9000  9050  9100  9150  9200  9250
  9300  9350  9400  9450  9500  9550  9600  9650  9700  9750  9800  9850
  9900 10000 10100 10200 10300 10400 10500 10600 10700 11000 11200 11500
 12000 12500 13000]
There are repititions in the CarrWu strikes
Final CarrWu Strikes: 
[8900 8900 8900 8900 8900 8900 8900 8900 8900 8900]
2020-06-25  Week:  0
['28D', '7D']
Theoeretical Strikes: 
[ 5877.4305875   6479.55503309  7045.8047707   7615.38037874
  8207.60671209  8838.34110241  9525.67359057 10295.7192424
 11195.4644085  12342.40484441]
Market Liquid Available Strikes: 
[ 9000  9100  9200  9300  9350  9400  9450  9500  9550  9600  9650  9700
  9750  9800  9900 10000 10100 10200]
There are repititions in the CarrWu strikes
Final CarrWu Strikes: 
[ 9000  9000  9000  9000  9000  9000  9550 10200 10200 10200]
2020-06-25  Week:  1
['21D', '7D']
The

In [1]:
# aa = np.array([1.2, 5.6, 10.2, 15.4])
# bb = np.array([2, 5, 12, 19, 20])
# aa_nearest = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]
# print(aa_nearest)


In [48]:
# 26000, 26500 -> 27000 -- 3000
# # print(gamma_with_spot(100, 100, 0.05, 0.5, 1))

In [ ]:
# generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident="BANKNIFTY")